# Simulating planetary orbits

## Introduction

The following code will be aiming to simulate the motion due to the gravitational force between two large objects. The simulation will aim to take into consideration the starting position, velocities and mass of the objects.
<br>Calculations will calculated using Newton's law of universal gravitation [Waugh]:
<br>(1): $F = \frac{Gm_1m_2}{r^2}$

-----------------------------
#### For the following code cell:
Importing the necassary python modules, numpy & vpython, to allow us to create our simulation. I will be setting any creating needed global variables here too.

In [1]:
# Imported modules
from vpython import vector, mag, sphere, rate, color, canvas
import numpy as np

# Global variables
G = 1.      # gravitational constant [using units where gravitational constant is 1.0]

<IPython.core.display.Javascript object>

## Part A

### 1. Calculating the gravitational force between two objects

You should complete the following function, without changing its name, arguments or docstring, to calculate the force on one object due to the gravitational field of another.

-----------------------------
#### For the following code cell:
I will be defining a function called 'force' that calculates the gravitational vector force exerted on object 1 due to object 2
<br>I will be using Newton's law of gravitation for this calculation:
<br>(1): $F = \frac{Gm_1m_2}{r^2}$
<br>Where:
<br>G = the gravitional constant (previously defined global variable). 
<br>m1 & m2 = masses of object 1 and 2 respectively
<br>r = distance between the two objects, from their centre of mass.
<br>
<br>(Following sourced from [Waugh])
<br>Since function will take in the starting parameters detailing the vector position of the two objects and their massess, 
<br>the r-value will be calculated as the magnitude of the vector from object 2's position vector to object 1's position vector ( Will be seen as 'posv_mag'). 
<br>This result will be multiplied by the unit vector of 'posv' (vector from object 2's position vector to object 1's position vector) so that function returns the force value in vector form.
<br>This gives us the following altered equation based on equation (1):
<br>(2): $F_{m_1m_2} = \frac{Gm_1m_2}{|r_{21}|^2} * \widehat{r_{21}}$ 


In [2]:
def force(pos1, pos2, m1, m2):
    """
    Returns the gravitational force exerted by object 2 on object 1.
    Input:
      - pos1 = position vector of first object
      - pos2 = position vector of second object
      - m1   = mass of first object
      - m2   = mass of second object
    Depends on:
      - G    = gravitational constant (global variable)
    """
    ##################
    # YOUR CODE HERE #
    ##################
    # Calculating r-value (posv_mag), vector magnitude between pos1 & pos2
    posv = pos1 - pos2
    posv_mag = np.sqrt(posv.dot(posv))
    
    # Calculating posv unit vector
    posv_hat = posv / posv_mag
    
    # Calculating gravitational vector force, Equ.(2)
    force = ( -(G * m1 * m2) / (posv_mag**2) ) * posv_hat
    return force

#### Testing your function

The following cell applies some tests to help you make sure your `force` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

In [3]:
################################################
#                                              #
# Test force is correct in a few simple cases. #
#                                              #
#   DO NOT CHANGE THE CODE IN THIS CELL.       #
#                                              #
################################################

def test_force(pos1, pos2, m1, m2, expected_force):
    epsilon = 1e-10
    f = force(pos1, pos2, m1, m2)
    if not isinstance(f,vector):
        print(f"ERROR: function should return a vector but returns {f}.")
        return
    args_as_string = f"({pos1}, {pos2}, {m1}, {m2})"
    error = mag(f-expected_force)
    if error<epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_force}")
        print(f"  got:      {f}")

test_force(vector(0,0,0),vector(1,0,0),1,1,vector(1,0,0))    # distance = 1 in x direction
test_force(vector(1,0,0),vector(0,0,0),1,1,vector(-1,0,0))   # swap objects
test_force(vector(0,0,0),vector(2,0,0),1,1,vector(0.25,0,0)) # distance = 2
test_force(vector(0,0,0),vector(0,1,0),1,1,vector(0,1,0))    # distance = 1 in y direction
test_force(vector(10,0,0),vector(10,1,0),1,1,vector(0,1,0))  # displaced from origin

OK: correct results for input (<0, 0, 0>, <1, 0, 0>, 1, 1)
OK: correct results for input (<1, 0, 0>, <0, 0, 0>, 1, 1)
OK: correct results for input (<0, 0, 0>, <2, 0, 0>, 1, 1)
OK: correct results for input (<0, 0, 0>, <0, 1, 0>, 1, 1)
OK: correct results for input (<10, 0, 0>, <10, 1, 0>, 1, 1)


### 2. Calculating the motion of a planet in the gravitational field of a star

You should complete the following function, without changing its name, arguments or docstring, to calculate the new position and velocity of a planet after a time step `dt`, in the gravitational field of a star of a given mass situated at the origin. This function will need to call `force` to calculate the acceleration vector of the planet.

------------------
#### For the following code cell :
I will be defining the function 'move_planet' that calculates the new position vector and velocity vector of a planet.
<br>The calculated changes will be based on the effects of a star's gravitational force, after a given time period, based on the equations derived from [Waugh].

<br>To simplify the calculation, we will be assuming that the star, due to it's considerably larger mass relative to the planet, will have negligable gravitational force/acceleration exerted on it.
<br>This allows us to take the position of the star as a fixed point since it will have negligiable motion.
<br>Adjusting Equ.(2) to account for this, we can then see $r_{21} = r_1 - r_2 $ turns into $r_{21} = r_1 - 0$ 

<br>This results in the following change to equation (2):
<br> $F_{m_1m_2} = \frac{Gm_1m_2}{|r_{1}|^2} * \widehat{r_{1}}$ 

<br>Which can then further simplified, knowing that $\widehat{r_{1}} = \frac{r_1}{|r_1|}$, to:
<br>$F_{m_1m_2} = \frac{Gm_1m_2 r_1}{|r_{1}|^3}$
<br>This means we don't need the initial vector position and vector velocity of the star to make the calculation.

<br>Next we must convert the equation to give us the acceleration of the planet due to the force, using Newton's second law: $F=ma$
<br>This results in the final equation that we will be using after dividing by $m_2$:
<br>$(3): a = \frac{Gm_1 r_1}{|r_{1}|^3}$

<br>Finally we can calculate the change in velocity and position with Eulers method, a simple and accurate enough method for our purposes.
<br>The velocity is calculated by adding the result of $ a * dt $ to the initial velcoity, a very small change in velocity, assuming $dt$ is also a very small time value.
<br>The position is calculated similarly, by adding the result of $ velocity * dt $ to the initial position, a very small change in position, assuming $dt$ is also a very small time value.
<br>(This avoids us using the differentials of acceleration, $ a = \frac{dv}{dt} = \frac{d^2 r}{dt}) $


In [4]:
def move_planet(position, velocity, m_star, dt):
    """
    Calculate motion of planet in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step
      - velocity: velocity vector of planet at start of time step
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step
      - velocity_new: velocity vector of planet at end of time step
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    ##################
    # YOUR CODE HERE #
    ##################
    # Calculating |r| for Equ.(3)
    position_mag = np.sqrt(position.dot(position))
    
    # Calculating acceleration using Equ.(3)
    a = (-G * m_star * position) / position_mag**3 
    
    # Calculating small change in velocity
    delta_v = a * dt 
    
    # Adding small change in velocity & position to initial conditions
    velocity_new = velocity + delta_v
    position_new = position + (velocity * dt)
    
    return position_new, velocity_new

#### Testing your function

The following cell applies some tests to help you make sure your `move_planet` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

In [5]:
######################################################
#                                                    #
# Test move_planet is correct in a few simple cases. #
#                                                    #
#   DO NOT CHANGE THE CODE IN THIS CELL.             #
#                                                    #
######################################################

def test_move_planet(position, velocity, m_star, dt, expected_pos, expected_vel):
    epsilon = 1e-10
    results = move_planet(position, velocity, m_star, dt)
    if not isinstance(results, tuple):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    if not len(results)==2:
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    pos, vel = results
    if not (isinstance(pos,vector) and isinstance(vel,vector)):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    args_as_string = f"{position}, {velocity}, {m_star}, {dt}"
    err_pos, err_vel = mag(pos - expected_pos), mag(vel - expected_vel)
    if err_pos < epsilon and err_vel < epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_pos, expected_vel}")
        print(f"  got:      {results}")


test_move_planet(vector(1,0,0), vector(1,0,0), 1, 0, vector(1,0,0), vector(1,0,0))   # dt = 0: output = input
test_move_planet(vector(1,0,0), vector(1,0,0), 1, 1, vector(2,0,0), vector(0,0,0))   # moving away from star
test_move_planet(vector(0,1,0), vector(0,-1,0), 1, 1, vector(0,0,0), vector(0,-2,0)) # moving towards star
test_move_planet(vector(1,0,0), vector(0,1,0), 1, 1, vector(1,1,0), vector(-1,1,0))  # moving past star
test_move_planet(vector(1,0,0), vector(0,1,0), 1, 0.1, vector(1,0.1,0), vector(-0.1,1,0))  # smaller dt
test_move_planet(vector(1,0,0), vector(0,1,0), 2, 0.1, vector(1,0.1,0), vector(-0.2,1,0))  # larger star mass

OK: correct results for input <1, 0, 0>, <1, 0, 0>, 1, 0
OK: correct results for input <1, 0, 0>, <1, 0, 0>, 1, 1
OK: correct results for input <0, 1, 0>, <0, -1, 0>, 1, 1
OK: correct results for input <1, 0, 0>, <0, 1, 0>, 1, 1
OK: correct results for input <1, 0, 0>, <0, 1, 0>, 1, 0.1
OK: correct results for input <1, 0, 0>, <0, 1, 0>, 2, 0.1


### 3. Animating the orbit of a planet

You should complete the following function, without changing its name, arguments or docstring, to display an animation of the orbit of the planet with the given intial position and velocity, and the given time step. Your planet should leave a visible trail so that we can see the shape of path.

--------------
#### For the following code cell:
I have defined the function 'animate_planet' which animates the motion of a planet due to the gravitational force of a star of known mass. 
<br>The function uses the previously defined 'move_planet' function to predict the orbit of the planet, using the 'position' and 'velocity' parameters to described intial conditions. It then loops at a rate based on given dt value to update and draw the new planet location, creating the animation.
<br>(Since we used the Euler method, I adapted code from module 9 coding task that also employed the Euler method )

In [6]:
def animate_planet(position, velocity, m_star, dt):
    """
    Animate planetary orbit from given starting position, with given time step.
    
    Input:
      - position: position vector of planet at start of time step
      - velocity: velocity vector of planet at start of time step
      - m_star:   mass of star
      - dt:       time step
      
    Output: Animation of predicted orbit of planet around star
    Depends:
      - 'move_planet' function
    """
    
    ##################
    # YOUR CODE HERE #
    #################################################
    # Adapted code from module 9 coding task
    # Changed functions and variables to be relevant.
    #################################################
    
    # Draw yellow planet at starting position, and make it leave a trail.
    planet = sphere(pos=position,radius=0.2,make_trail=True,color=color.yellow)
    
    # Setting the conditions for while loop
    time = 0.                               # initial time [s]
    max_time = 10.                          # maximum time to run animation
    current_velocity = velocity

    # While loop that updates planet's position and velocity depending on input
    while time < max_time:
        rate(1/dt)
        position, velocity = move_planet(position, current_velocity, m_star, dt)
        planet.pos = position
        current_velocity = velocity
        time += dt 

#### Testing your function

If you have implemented all three functions correctly, the following cell should show an almost elliptical orbit.

In [7]:
# Initialize canvas, and set parameters of star and planet.
canvas()
pos_planet = vector(0,2,0)   # initial position of planet
v_planet   = vector(-22,0,0) # initial velocity of planet
m_star     = 1000.           # mass of star (units where G=1)
sphere(pos=vector(0,0,0), color=color.yellow, radius=0.1) # draw star

# Animate orbit of planet 
animate_planet(pos_planet, v_planet, m_star, 1e-4)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 4. Investigation

You should insert code and text cells below as required to investigate and discuss the effect of changing the parameters of the animation: time step, mass of star, initial position and velocity of planet. 

-------------------------
#### For the following code cell:
I have created the 'draw_scene' function to help draw the environment of the scene.
<br>I also have created a list of default parameters that will be used throughout the animations, allowing the changes to more visible. 
<br>The values are based on the animation test function so that I can use the animation as a point of comparison.

In [8]:
# Drawing environment for orbits
def draw_scene():
    canvas()
    sphere(pos=vector(0,0,0), color=color.yellow, radius=1) # draw star

# List of default parameters
pos_planet = vector(0,2,0)   # initial position of planet
v_planet   = vector(-22,0,0) # initial velocity of planet
m_star     = 1000.           # mass of star (units where G=1)
time_step  = 1e-4            # default time step, dt, value

-----------------
## Changing mass of star investiation (A - B)

-----------------
#### For the following code cell (A):
I have increased the star mass used in the animation from 1000 to 2000, keeping the rest of the parameters the same as the test animation.

In [9]:
draw_scene()
animate_planet(pos_planet, v_planet, 2000, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (B):
I have decreased the star mass used in the animation from 1000 to 500, keeping the rest of the parameters the same as the test animation.

In [10]:
draw_scene()
animate_planet(pos_planet, v_planet, 500, time_step)

<IPython.core.display.Javascript object>

## Investigation conclusion on changing mass of star:
We see that when the mass is increased (Code Cell A) from our test animation starting values, we see that the planet's orbit becomes more obviously elliptical in comparision to the more circular orbit of the test.
<br>We see that when the mass is decreased (Code Cell B), the planet is lauched into an almost trajectory like path and doesn't even complete a full orbit.

These orbits could be explained by how the gravitational force experienced by the planet is proportional to the mass of the star.
<br>Smaller force means the planet gets closer to the possibility of escaping the the gravitational attraction of the star just from it's initial velocity.
<br>Larger force means the planet experiences high levels of force when it approaches closer during it's orbit cycle. This increase in acceleration/ speed causes it to 'slingshot' back round, slowing down until it starts accelerating back again. This explains the why the orbit gets more elliptical and have a higher number of revolutions.


-----------------
## Changing initial velocity investiation (C - F)

-----------------
#### For the following code cell (C):
I have changed the initial velocity of the planet used in the animation (-22,0,0) to (-30,0,0), keeping the rest of the parameters the same as the test animation.

In [11]:
draw_scene()
animate_planet(pos_planet,  vector(-30,0,0), m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (D):
I have changed the initial velocity of the planet used in the animation from (-30,0,0) to (-50,0,0), keeping the rest of the parameters the same as the test animation.

In [12]:
draw_scene()
animate_planet(pos_planet,  vector(-50,0,0), m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (E):
I have changed the initial velocity of the planet used in the animation from (-22,0,0) to (15,0,0), keeping the rest of the parameters the same as the test animation.

In [13]:
draw_scene()
animate_planet(pos_planet,  vector(15,0,0), m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (F):
I have changed the initial velocity of the planet used in the animation from (-22,0,0) to (-22,0,22), keeping the rest of the parameters the same as the test animation.

In [14]:
draw_scene()
animate_planet(pos_planet, vector(-22,0,22), m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (F2):
I have changed the initial velocity of the planet used in the animation from (-22,0,0) to (0,0,-22), keeping the rest of the parameters the same as the test animation.

In [15]:
draw_scene()
animate_planet(pos_planet, vector(0,0,-22), m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (F3):
I have changed the initial velocity of the planet used in the animation from (-22,0,0) to (0,2,0), keeping the rest of the parameters the same as the test animation.

In [16]:
draw_scene()
animate_planet(pos_planet, vector(0,2,0), m_star, time_step)

<IPython.core.display.Javascript object>

## Investigation conclusion on changing initial velocity:

When initial velocity is increased slightly (code cell C), it results in the planet completing a more elliptical orbit compared to the test animation. This could be explained by how with a larger initial velocity, the more kinetic energy the planet has to work against the gravitational force of the star. It thus is able to travel further away from the star, hence the more elliptical orbits. When it reachs a velocity of 0 the first time, it starts to accelerate according to the gravitational force of the star, returning closer to the star.

<br>When initial velocity is increased a significantly (code cell D), it results in the planet moving in a straight line in the direction of the velocity. This could be explained by how with a large enough initial velocity, the planet has enough energy to work against the gravitational force of the star and escape it's influence to continue travelling in the direction of the velocity with left over energy.

<br>When initial velocity direction is changed (code cell E), oribit is changes rotatation direction from anti-clockwise (negative) to clockwise (positive). This possibly shows that the gravitational force has little to no influence on which way a planet orbits the star, it appears to rely more on the initial velocity of the planet.

<br>Having additional  z-component (code cell F) of the same magnitude causes a similar result to code cell D. Planet moves in a straight line in the direction of the velocity (Now a combination of x & z components.). 

<br>Changing having only z-component of the velocity (code cell F2) changes which plane of space the orbit is seen rotating in. his possibly shows that the gravitational force has little to no influence on which plane a planet orbits the star, it appears to rely more on the initial velocity of the planet.

<br>Having only the y-component produces odd result of planet taking on a straight path, even with small magnitudes. This shows potentially issues with my code, that it may require further calculations for accurate representation of y-component. I believe this issue could be a result of the planet getting too close to the star and 'experiencing' very large amounts of gravitational acceleration, thus getting launched away. 


-----------------
## Changing initial position investigation (G - I)


-----------------
#### For the following code cell (G):
I have changed the initial position of the planet used in the animation from (0,2,0) to (0,0,2), keeping the rest of the parameters the same as the test animation.

In [17]:
draw_scene()
animate_planet(vector(0,0,2), v_planet, m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (H):
I have changed the initial position of the planet used in the animation from (0,2,0) to (2,0,0), keeping the rest of the parameters the same as the test animation. Tested another animation for initial position of (100,0,0) for further investigation

In [18]:
draw_scene()
animate_planet(vector(2,0,0), v_planet, m_star, time_step)
animate_planet(vector(100,0,0), v_planet, m_star, time_step)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (I):
I have changed the initial position of the planet used in the animation from (0,2,0) to (0,4,0), keeping the rest of the parameters the same as the test animation.

In [19]:
draw_scene()
animate_planet(vector(0,4,0), v_planet, m_star, time_step)

<IPython.core.display.Javascript object>

## Investigation conclusion on changing initial position:

When only z-component has a magnitude, the orbital changes it's plane it orbits around. (code cell G) This shows that the gravitational force has little to no influence in the orbital plane and that it likely mostly depends on initial positio and velcoity direction.

<br>When only x-component has a magnitude, the planet takes a straight path away from the star similar to planet paths seen in code cells F3, D. Testing for large values of x, 100, we see in time what is taking place. The planet moves with default velocity directly towards the planet. It passes through the 'star' accelerating exponentially when approaching. It then speeds through the star in the original direction it was going. This shows us the limitation of our modelling as it doesn't consider the case of collision. This helps explain the cases for F3 and D.

<br>When the y-component increases, it causes the distance away from the star increase and more elliptical/ parabolic trajectory occurs. This can be explained by the reduced effects of the star's gravitational attraction due to the extra distance. It causes the planet to only have it's path altered but not enough for it to start orbiting. 

<br>With mixture of the different components, we see a mixture of the individual effects. Larger magnitude of the component, the more prevalent it is. Main difference is that the x-component can have a magnitude likel it no longer is on path to 'collide' with the star with other positional vectors.

-----------------
## Changing timestep investiation (J - K)

-----------------
#### For the following code cell (J):
I have changed the time step used in the animation from 1e-4 to 0.001, keeping the rest of the parameters the same as the test animation.

In [20]:
draw_scene()
animate_planet(pos_planet, v_planet, m_star, 0.001)

<IPython.core.display.Javascript object>

-----------------
#### For the following code cell (k):
I have changed the time step used in the animation from 1e-4 to 0.1, keeping the rest of the parameters the same as the test animation.

In [21]:
draw_scene()
animate_planet(pos_planet, v_planet, m_star, 0.1)

<IPython.core.display.Javascript object>

#### For the following code cell (L):
I have changed the time step used in the animation from 1e-4 to 0.001, keeping the rest of the parameters the same as the test animation.

In [ ]:
draw_scene()
animate_planet(pos_planet, v_planet, m_star, 0.000001)

<IPython.core.display.Javascript object>

## Investigation conclusion on changing timestep:

We have increased our timestep from 1e-4 to 0.001, as seen in code cell J.
Noticably, the number of lines drawn is less. Using 0.001 actually makes it easier to see the orbit increase in size. Both are unable to update fast enough to smoothly model the speed at which this orbits goes. We see that with slower rate of updating, the animation becomes more choppy and less accurate.

<br>We have increased our timestep from 1e-4 to 0.1, as seen in code cell K. The simuluation doesn't model a circular/elliptical orbit that is expected. It creates a straight line. The rate of updating has reached a point of total inaccurancy as the model doesn't create a meaningful result. 

<br>I decreased the timestep from 1e-4 to 0.000001, as seen in code cell L. The simulation models the circular orbit very smoothly. It however took a much longer time to compute as more calculations are needed. 

I have tried decreasing our timestep further, however due to the limitation of Eulers method addressed in module 9 coding task, jupyter is unable to compute it in a feasible time frame. This shows us how there is room for improvement.

-----------------------------------

## Part B

It is up to you to structure the rest of the notebook as you see fit, as you complete the tasks set in part B of the assignment.

You can call the functions you have defined in part A, or copy and adapt them in part B, but **DO NOT CHANGE THE CODE IN PART A SUCH THAT THE CELLS IN PART A NO LONGER WORK CORRECTLY.** You do not want to lose marks in section A in completing section B.

# Two planets (20% of marks)
## 1a)
"Copy the animate planet function from part A into a new code cell, and
adapt it so that instead of a single planet it shows the paths of two separate
planets in the gravitational field of the same star. 
<br>You should not take into
account the gravitational attraction between the planets in this section, only
the force between each planet and the star.
<br>Instead of a single position and velocity, your function will need to take as
arguments two positions and velocities (a position and velocity for each
planet) and return two positions and velocities.
<br>Your function can call move planet from part A, and hence make use of the
existing force function. You do not need to repeat or change these functions."

--------------------------
#### For the following code cell:
<br>Adapting previously coded 'animate planet' function to now consider and model the orbits of two planets around a star.
<br>It now takes additional arguments 'position2' and 'velocity2' that allow the second planet to have different starting conditions.
<br>The calculations of the change in position and velocity will still be based solely on the 'move_planet' function.
<br>Thus we must note the lack of consideration to the gravitational force between the two planets.


In [ ]:
###############################################################################
# Adapted 'animate_planet' function, which was adapted from module 9 coding task
# Added the consideration of a second planet
###############################################################################
def animate_2_planets(position, velocity,position2, velocity2, m_star, dt):
    """
    Animate planetary orbit from given starting position, with given time step.
    
    Input:
      - position: position vector of planet at start of time step
      - velocity: velocity vector of planet at start of time step
      - position2: position vector of 2nd planet at start of time step
      - velocity2: velocity vector of 2nd planet at start of time step
      - m_star:   mass of star
      - dt:       time step
      
    Output: Animation of predicted orbit of two planets around a star
    
    Depends on:
      - 'move_planet' function
    """
    # Draw yellow and green planet at starting position, and make it leave a trail.
    planet = sphere(pos=position,radius=0.2,make_trail=True,color=color.yellow)
    planet2 = sphere(pos=position,radius=0.2,make_trail=True,color=color.green)
    
    # Setting the conditions for while loop
    time = 0.                               # initial time [s]
    max_time = 10.                          # maximum time to run animation
    current_velocity = velocity
    current_velocity2 = velocity2

    # While loop that updates both planet's position and velocity depending on input
    while time < max_time:
        rate(1/dt)
        position 
        position, velocity = move_planet(position, current_velocity, m_star, dt)
        position2, velcoity2 = move_planet(position2, current_velocity2, m_star, dt)
        planet.pos = position
        planet2.pos = position2
        current_velocity = velocity
        current_velocity2 = velocity2
        time += dt 

## 1b)
Test your code using the following initial conditions:
<br>• first planet: position (1.85, 0, 0), velocity (0, 23.25, 0)
<br>• second planet: position (1.96, 0.4, 0), velocity (−4.44, 21.9, 0)
<br>Describe and comment on the behaviour in a text cell.


-------------------
#### For the following code cell:
Using 'animate_2_planets' and 'draw_scene' function to animate the initial conditions given for testing.

In [ ]:
draw_scene()
animate_2_planets(vector(1.85,0,0), vector(0,23.25,0),vector(1.96,0.4,0),vector(-4.44,21.9,0),1000,0.0001)

## Description and comments on test animation.

We see that the yellow planet orbits the star that increases in diameter over time. It likely did not have high enough inital velocity to escape gravitational attraction of the star. 
We see that the green planet may have had high enough velocity to escape the gravitational attraction of the star, having it's path adjusted slightly towards the star but still continuing on a straight away from the star. 


# Two planets with gravitational interactions (30% of marks)
## 2a) 
Create a new function, similar to move planet, but this time dealing with two
planets and taking into account the gravitational attraction between the
planets as well as between each planet and the star.
<br>In addition to the positions and velocities of the two planets, this function will
need to take two more arguments to represent the masses of the planets.
<br>Your function can call the force function from part A: you do not need to
repeat or change this function.


-----------------------------
#### For the following code cell:
I have defined the function 'move_2_planet' adapted from the 'move_planet' function.
<br>This function calculates the new position and velocity of two planets due to the effects of gravitational forces.
<br>It takes into consideration the effect of the gravitational force the planets have on each other.
<br>It takes into consideration the effect of the gravitational force that the star exerts on the planets.

<br>Similar to the explanation seen for 'move_planet' function, this function adds a very small change in velcoity due to calculating the acceleration for a small time period, 'dt'. Main difference is that this function adds the small change in velocity of a planet due to the gravitational force of the other planet. 


In [ ]:
##################################################################################
# Adapted from 'move_planet' function defined earlier
# Now computes additional planet, taking into account their effects on each other
##################################################################################
def move_2_planet(position, velocity, m_planet, position2, velocity2, m_planet2, m_star, dt):
    """
    Calculate motion of 2 planets in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step
      - velocity: velocity vector of planet at start of time step
      - position2: position vector of 2nd planet at start of time step
      - velocity2: velocity vector of 2nd planet at start of time step
      - m_planet: mass of the planet
      - m_planet2: mass of the second planet
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step
      - velocity_new: velocity vector of planet at end of time step
      - position2_new: position vector of 2nd planet at end of time step
      - velocity2_new: velocity vector of 2nd planet at end of time step
      
    Depends on:
      - 'force' function
      - 'move_planet' funtion
    """
    ##################
    # YOUR CODE HERE #
    ##################
    
    # Calculating distance between planets, 'planets_d'
    dot_product = abs(position.dot(position2))
    planets_d = np.sqrt(dot_product)
    
    # Calculating gravitational vector force experienced by the planets due to each other
    g_force = force(position, position2, m_planet, m_planet2)
    g_force2 = force(position2, position, m_planet2, m_planet)
    
    # Calculating gravitational acceleration experienced by the planets due to each other
    a = g_force / m_planet
    a2 = g_force2 / m_planet2
    
    # The additional velocity due to gravitational acceleration for each planet for time dt
    delta_v = a * dt
    delta_v2 = a2 * dt
    
    # The initial velocity + additional velocity due to gravitational force of the star for each planet 
    star_p, star_v = move_planet(position,velocity, m_star, dt)
    star_p, star_v2 = move_planet(position2,velocity2, m_star, dt)
    
    # The total change in velocity calculations 
    velocity_new = star_v + delta_v
    velocity2_new = star_v2 + delta_v2
    
    # The total change in position calculations
    position_new = position + (velocity_new * dt) 
    position2_new = position2 + (velocity2_new * dt)
    
    return position_new, velocity_new, position2_new, velocity2_new

## 2b)
Create a new function, similar to animate planet, but this time dealing with
two planets and using your new move planet function to take into account
the gravitational attraction between the planets as well as between each
planet and the star.


---------------------
#### For the following code cell:
I have defined the function 'new_animate_2_planets' which animates the motion of a planets due to the gravitational of a star and each other. Adapted from animate_planet & animate_2_planets, the key difference being that it now depends on the 'move_2_planets' function. 

<br>The function loops through 'move_2_planets' starting from the values inputted into it at a rate of 'dt'. It continues to update and draw both the planet locations, resulting in an animation.
(Also uses Euler's method, hence why I adapted animate_planet & animate_2_planets functions which were adapted from module 9 coding task )

## 2c)
Test your code using the same initial conditions as for the previous case, and
with each planet having a mass of 2.0 in the units we are using for this
assignment.
Describe and comment on the behaviour in a text cell

In [ ]:
####################################################################################################
# Adapted 'animate_2_planet' function, which was adapted from 'animate_planet'/module 9 coding task
# Now depends on 'move_2_planet' function instead of 'move_planet' function
####################################################################################################
def new_animate_2_planets(position, velocity, m_planet, position2, velocity2, m_planet2, m_star, dt):
    """
    Animate planetary orbit from given starting position, with given time step.
    
    Input:
      - position: position vector of planet at start of time step
      - velocity: velocity vector of planet at start of time step
      - m_planet: mass of planet
      - position2: position vector of 2nd planet at start of time step
      - velocity2: velocity vector of 2nd planet at start of time step
      - m_planet2: mass of 2nd planet
      - m_star:   mass of star
      - dt:       time step
      
    Output: Animation of predicted orbit of two planets around a star
    
    Depends on:
      - 'move_2_planet' function
    """
    ##################
    # YOUR CODE HERE #
    #####################
    # Taken from module 9
    #####################
    
    # Draw yellow & green planets at starting position, and make it leave a trail.
    planet = sphere(pos=position,radius=0.2,make_trail=True,color=color.yellow)
    planet2 = sphere(pos=position,radius=0.2,make_trail=True,color=color.green)
    
    # Setting conditions needed for while loop
    time = 0.                               # initial time [s]
    max_time = 10.                          # maximum time to run animation
    current_velocity = velocity
    current_velocity2 = velocity2
    
    # While loop that updates
    while time < max_time:
        rate(1/dt)
        position, velocity, position2, velocity2 = move_2_planet(position, current_velocity, m_planet, position2, current_velocity2, m_planet2, m_star, dt)
        planet.pos = position
        planet2.pos = position2
        current_velocity = velocity
        current_velocity2 = velocity2
        time += dt 
    

In [ ]:
draw_scene()
new_animate_2_planets(vector(1.85,0,0), vector(0,23.25,0), 2, vector(1.96,0.4,0),vector(-4.44,21.9,0), 2, 1000, 0.001)

## Description and comments on test animation

We see that both planets orbit the star at different speeds. This results in many points where they influence each others orbits and cause it to move more elliptically. This is seen more in the yellow planet. Finally the planets fall into an equilibirum where they consistently orbit along the same path. Potentially, this is the orbit's initial kinetic energy causing the initial instability, once both planets have their kinetic energies expended, they fall into the equilibrium.

# Further investigation
## 3.)
Try varying some of the parameters of your simulation: the masses of the star and
the planets, and the starting positions and velocities of the planets. 
<br>There are
many aspects you could investigate, but you should pick one or two. 
<br>Create a few
animations to illustrate some of the effects you observe, and write a conclusion
describing the behaviour and attempting to explain it.
<br>For example, you could
<br>• calculate the required velocity to achieve a stable circular orbit with a given
radius, and see how close or how massive the planets have to be to have a
significant effect on each other’s orbit;
<br>• turn one of the planets into a moon orbiting the other, by careful choice of
masses and initial conditions. (This is hard.)


--------------------
#### For the following code cell:
I have experimented with changing the variables that input into the 'new_animate_2_planets' function so that it now animates a moon orbiting a planet, both orbiting the star.

In [ ]:
# Parameters to be inputted into animation function 
earth_pos = vector(0,3.5,0)  
earth_v = vector(-22,0,0)
earth_m = 2
moon_pos = vector(0,3,0) 
moon_v = vector(-25,0,0)
moon_m = 0.5
m_star = 1000
timestep =  0.00001

# Drawing and executing animation function 
draw_scene()
new_animate_2_planets(moon_pos, moon_v , moon_m, earth_pos,earth_v , earth_m , m_star, timestep)

## Final Investigation process

Initially, I based the model on the actual masses and distances between the Sun, Earth and the Moon. However due to limitations of scaling it to be visible, I quickly moved on to basing the model on the circular orbit seen in the test animation. (0,2,0) & (0,2.5,0) Being the positional vector of the planets, (-22,0,0) being initial velocity of both planet, 2 being the mass of both plants and mass of tje star being 1000.    

First I adjusted the positions of the two planets so that they would be start further away from the star. This was so that they would experience weaker amounts gravitational attraction and be able to have more influence on each other. (keeping the distance between the planets small too so that it's easier for a moon orbit)

Next I increased the velocity of the moon planet so that it would follow a similar trajectory to the planet despite it being closer and experiencing stronger gravitational acceleration.

Finally I reduced the mass of the moon planet relative to the planet so that it could experience significant gravitational accleration from the planet for it to orbit. I used the ratio of the Earth and Moon for reference [Williams]

What we observe is the planet and moon following similar trajectories until the moon starts to orbit the planet (green). The yellow spiral trail following the circular green trail shows the moon to planet behaviour that we wanted. 






## References 

Waugh, Dr Ben, et al. “Simulating Planetary Orbits.” Moodle.Ucl.Ac.Uk, UCL, 16 Dec. 2020, moodle.ucl.ac.uk/pluginfile.php/3716400/mod_resource/content/1/PHAS0007_Final_Assignment_2020-1.pdf.

Williams, Dr. David R. “Planetary Fact Sheet - Ratio to Earth.” Nasa.Gov, NASA, 2019, nssdc.gsfc.nasa.gov/planetary/factsheet/planet_table_ratio.html.